In [ ]:
import glob
import os
import scipy
from scipy.optimize import minimize
from abc import ABC
from abc import abstractmethod

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

In [ ]:
files = glob.glob('data/*.csv')
files

In [ ]:
file = 'data/global-reit.csv'
column = os.path.basename(file).split('.')[0]
global_reit = pd.read_csv(file, parse_dates=['基準日']).rename(columns={'基準日': 'date'}).set_index('date')
global_reit['分配金'] = global_reit['分配金'].str.replace('-', '0').map(float)
global_reit[column] = global_reit['基準価額'] + global_reit['分配金']
global_reit = global_reit[[column]]

In [ ]:
file = 'data/japan-reit.csv'
column = os.path.basename(file).split('.')[0]
japan_reit = pd.read_csv(file, parse_dates=['基準日']).rename(columns={'基準日': 'date'}).set_index('date')
japan_reit['分配金'] = japan_reit['分配金'].str.replace('-', '0').map(float)
japan_reit[column] = japan_reit['基準価額'] + japan_reit['分配金']
japan_reit = japan_reit[[column]]

In [ ]:
file = 'data/global-equity.csv'
column = os.path.basename(file).split('.')[0]
global_equity = pd.read_csv(file, encoding='shift-jis', header=1, parse_dates=['基準日']).rename(columns={'基準日': 'date'}).set_index('date')
global_equity = global_equity[['基準価額（分配金再投資）(円)']].rename(columns={'基準価額（分配金再投資）(円)': column})

In [ ]:
file = 'data/japan-equity.csv'
column = os.path.basename(file).split('.')[0]
japan_equity = pd.read_csv(file, encoding='shift-jis', header=1, parse_dates=['基準日']).rename(columns={'基準日': 'date'}).set_index('date')
japan_equity = japan_equity[['基準価額（分配金再投資）(円)']].rename(columns={'基準価額（分配金再投資）(円)': column})

In [ ]:
file = 'data/global-bond.csv'
column = os.path.basename(file).split('.')[0]
global_bond = pd.read_csv(file, encoding='shift-jis', header=1, parse_dates=['基準日']).rename(columns={'基準日': 'date'}).set_index('date')
global_bond = global_bond[['基準価額（分配金再投資）(円)']].rename(columns={'基準価額（分配金再投資）(円)': column})

In [ ]:
file = 'data/japan-bond.csv'
column = os.path.basename(file).split('.')[0]
japan_bond = pd.read_csv(file, encoding='shift-jis', header=1, parse_dates=['基準日']).rename(columns={'基準日': 'date'}).set_index('date')
japan_bond = japan_bond[['基準価額（分配金再投資）(円)']].rename(columns={'基準価額（分配金再投資）(円)': column})

In [ ]:
df = pd.concat([global_equity, japan_equity, global_bond, japan_bond], axis=1).dropna().sort_index()
# df = pd.concat([global_reit, japan_reit, global_equity, japan_equity, global_bond, japan_bond], axis=1).dropna().sort_index()

df = df.reset_index().assign(yearmonth=lambda d: d['date'].dt.strftime('%Y-%m')).groupby('yearmonth').last().set_index('date').pct_change().dropna() * 100
df.columns, len(df)

In [ ]:
class Simulation(ABC):
    def __init__(self, initial_weight, data):
        self.initial_weight = initial_weight
        self.data = data
        self.cov = data.cov().to_numpy()
        self.eigenvalues, self.eigenvectors = np.linalg.eig(self.cov)
        
    def compute_information(self, weight):
        weight_pc = self.eigenvectors.T @ weight
        variances = weight_pc * weight_pc * self.eigenvalues
        probs = variances / variances.sum()
        information = -probs * np.log(probs)
        return information
    
    def compute(self):
        weight = self.compute_weight()
        return {'weight_asset': weight, 'cov': self.cov, 'eigenvalues': self.eigenvalues, 'eigenvectors': self.eigenvectors, 'information': self.compute_information(weight)}
    
    @abstractmethod
    def compute_weight(self):
        pass

In [ ]:
class SimulationSameWeight(Simulation):
    def compute_weight(self):
        return np.ones(len(self.data.columns)) / len(self.data.columns)

In [ ]:
class SimulationRiskParity(Simulation):
    def compute_weight(self):
        sigmas = np.sqrt(np.diag(self.cov))
        weight = 1 / sigmas
        weight = weight / weight.sum()
        return weight

In [ ]:
class SimulationBenchmark(Simulation):
    def compute_weight(self):
        name_to_weight = {'global-reit': 0.0, 'japan-reit': 0.0, 'global-equity': 0.1, 'japan-equity': 0.3, 'global-bond': 0.2, 'japan-bond': 0.4}
        return df.columns.map(lambda c: name_to_weight[c]).to_numpy()

In [ ]:
class SimulationMinimumVariance(Simulation):
    def compute_weight(self):
        result = minimize(self.compute_variance, self.initial_weight, constraints={'type': 'eq', 'fun': lambda x: 1 - np.sum(x)}, bounds=[(0, 1)] * len(self.data.columns))
        return result.x

    def compute_variance(self, weight):
        variance = weight @ self.cov @ weight
        return variance

In [ ]:
class SimulationPcaVariance(Simulation):
    def compute_weight(self):
        result = minimize(self.compute_neg_entropy, self.initial_weight, constraints={'type': 'eq', 'fun': lambda x: 1 - np.sum(x)}, bounds=[(0, 1)] * len(self.data.columns))
        return result.x

    def compute_neg_entropy(self, weight):
        entropy = self.compute_information(weight).sum()
        return -np.exp(entropy)

In [ ]:
class SimulationPcaKde(Simulation):
    def compute_weight(self):
        result = minimize(self.compute_neg_entropy, self.initial_weight, constraints={'type': 'eq', 'fun': lambda x: 1 - np.sum(x)}, bounds=[(1e-10, 1)] * len(self.data.columns))
        return result.x

    def compute_neg_entropy(self, weight):
        noise_data = np.random.normal(0, 1e-10, self.data.shape).T
        arg = (data.to_numpy() * weight).T + noise_data
        
        noise_prob = np.abs(np.random.normal(0, 1e-10, arg.shape[1]).T)
        prob = np.abs(scipy.stats.gaussian_kde(arg)(data.to_numpy().T)) + noise_prob
        prob = prob / prob.sum()
        entropy = -np.sum(prob * np.log(prob))
        return -entropy

In [ ]:
def compute(sim: Simulation):
    result = sim.compute()
    weight_pc = sim.eigenvectors.T @ result['weight_asset']
    summary = pd.DataFrame().assign(column=df.columns).assign(date=date).assign(weight=result['weight_asset']).assign(risk=np.sqrt(np.diag(result['cov']) * result['weight_asset'] * result['weight_asset'])).assign(risk_pc=np.sqrt(result['eigenvalues'] * weight_pc * weight_pc)).assign(pc1_ratio=lambda d: d['risk_pc'].max() / d['risk_pc'].sum()).assign(information=result['information']).assign(pl=(result['weight_asset'] * today).tolist()).assign(total_std=np.sqrt(result['weight_asset'] @ result['cov'] @ result['weight_asset'])).set_index(['date', 'column'])
    for i in range(len(df.columns)):
        summary['eigenvector_' + str(i)] = sim.eigenvectors[i]
    return summary

In [ ]:
df.index.min(), df.index.max()

In [ ]:
size = 60
dates = list(df.index)[size:]
summaries = []

for date in dates:
    print(date)
    data = df[df.index < date].tail(size)
    today = df[df.index == date].iloc[0]
    summaries.append(compute(SimulationPcaVariance(np.ones(len(df.columns)) / len(df.columns), data)).assign(method='pca_variance'))
    # summaries.append(compute(SimulationPcaKde(np.ones(len(df.columns)) / len(df.columns), data)).assign(method='pca_kde'))
    summaries.append(compute(SimulationRiskParity(np.ones(len(df.columns)) / len(df.columns), data)).assign(method='risk_parity'))
    summaries.append(compute(SimulationBenchmark(np.ones(len(df.columns)) / len(df.columns), data)).assign(method='pension_fund'))
    summaries.append(compute(SimulationMinimumVariance(np.ones(len(df.columns)) / len(df.columns), data)).assign(method='minimum_variance'))
    summaries.append(compute(SimulationSameWeight(np.ones(len(df.columns)) / len(df.columns), data)).assign(method='same_weight'))

In [ ]:
result = pd.concat(summaries).reset_index()

In [ ]:
result

In [ ]:
summation = result.drop(columns=['column']).groupby(['date', 'method']).sum()
summation = summation.groupby('method').pl.describe()
summation['return (yearly)'] = summation['mean'] * 12
summation['std (yearly)'] = summation['std'] * np.sqrt(12)
summation['return / std (yearly)'] = summation['return (yearly)'] / summation['std (yearly)']

# save df to fig
summation[['return (yearly)', 'std (yearly)', 'return / std (yearly)']]

In [ ]:
tmp = df[df.index.isin(result.date.unique())]
tmp = pd.concat([tmp[[col]].assign(asset=col).rename(columns={col: "pl"}) for col in tmp.columns])

summation = tmp.groupby('asset').pl.describe()
summation['return (yearly)'] = summation['mean'] * 12
summation['std (yearly)'] = summation['std'] * np.sqrt(12)
summation['return / std (yearly)'] = summation['return (yearly)'] / summation['std (yearly)']

summation[['return (yearly)', 'std (yearly)', 'return / std (yearly)']]

In [ ]:
tmp = df[df.index.isin(result.date.unique())].to_numpy()
cov = np.cov(tmp.T)
eigenvalues, eigenvectors = np.linalg.eig(cov)

tmp = tmp @ eigenvectors
tmp = pd.DataFrame(tmp, columns=[f'pc{i+1}' for i in range(len(df.columns))])
tmp = pd.concat([tmp[[col]].assign(pc=col).rename(columns={col: "pl"}) for col in tmp.columns])

summation = tmp.groupby('pc').pl.describe()
summation['return (yearly)'] = summation['mean'] * 12
summation['std (yearly)'] = summation['std'] * np.sqrt(12)
summation['return / std (yearly)'] = summation['return (yearly)'] / summation['std (yearly)']

summation[['return (yearly)', 'std (yearly)', 'return / std (yearly)']]

In [ ]:
methods = result.method.unique()
for method in methods:
    print('weight', method)
    tmp = result[result.method == method].groupby(['date', 'column']).sum().reset_index().sort_values('date')
    tmp = tmp.pivot(index='date', columns='column', values='weight').fillna(0)
    for col in df.columns: tmp[col] = tmp[col].abs()

    tmp.plot(kind='bar', stacked=True, figsize=(10, 4), title=method)
    plt.show()
    
    tmp.plot(kind='bar', stacked=True, figsize=(10, 4), title=method)
    plt.savefig(f'fig/weight_{method}.png')
    plt.close()

In [ ]:
methods = result.method.unique()
for method in methods:
    print('risk', method)
    tmp = result[result.method == method].groupby(['date', 'column']).sum().reset_index().sort_values('date')
    tmp = tmp.pivot(index='date', columns='column', values='risk').fillna(0)
    for col in df.columns: tmp[col] = tmp[col].abs()

    tmp.plot(kind='bar', stacked=True, figsize=(10, 4), title=method)
    plt.show()

    tmp.plot(kind='bar', stacked=True, figsize=(10, 4), title=method)
    plt.savefig(f'fig/risk_{method}.png')
    plt.close()

In [ ]:
methods = result.method.unique()
for method in methods:
    print('risk_pc', method)
    tmp = result[result.method == method].groupby(['date', 'column']).sum().reset_index().sort_values('date')
    tmp = pd.concat([grp.sort_values('risk_pc', ascending=False).assign(column=[f'risk_pc_{i+1}' for i in range(len(df.columns))]) for by, grp in tmp.groupby(['date'])])
    tmp = tmp.pivot(index='date', columns='column', values='risk_pc').fillna(0)

    tmp.plot(kind='bar', stacked=True, figsize=(10, 4), title=method)
    plt.show()

    tmp.plot(kind='bar', stacked=True, figsize=(10, 4), title=method)
    plt.savefig(f'fig/risk_pc_{method}.png')
    plt.close()

In [ ]:
methods = result.method.unique()
for method in methods:
    print('information', method)
    tmp = result[result.method == method].groupby(['date', 'column']).sum().reset_index().sort_values('date')
    tmp = tmp.pivot(index='date', columns='column', values='information').fillna(0)
    for col in df.columns: tmp[col] = tmp[col].abs()

    tmp.plot(kind='bar', stacked=True, figsize=(10, 4), title=method)
    plt.show()

    tmp.plot(kind='bar', stacked=True, figsize=(10, 4), title=method)
    plt.savefig(f'fig/information_{method}.png')
    plt.close()

In [ ]:
tmp = result.groupby(['date', 'method']).sum()[['information']].assign(div_index=lambda d: d['information'].map(np.exp)).reset_index()
tmp = tmp[['date', 'method', 'div_index']].pivot(index='date', columns='method', values='div_index').fillna(0)

tmp.plot(kind='line', stacked=False, figsize=(10, 4), title='div_index')
plt.show()

tmp.plot(kind='line', stacked=False, figsize=(10, 4), title='div_index')
plt.savefig(f'fig/div_index.png')

In [ ]:
methods = result.method.unique()
for method in methods:
    print('pc1_ratio', method)
    tmp = result[result.method == method].drop(columns=['column'])[['date', 'pc1_ratio']].groupby(['date']).mean().reset_index().sort_values('date').set_index('date')
    

    tmp.plot(kind='line', stacked=True, figsize=(10, 4), title=method)
    plt.show()

    tmp.plot(kind='line', stacked=True, figsize=(10, 4), title=method)
    plt.savefig(f'fig/pc1_ratio_{method}.png')
    plt.close()

In [ ]:
methods = result.method.unique()
for method in methods[:1]:
    tmp = result[result.method == method][['date', 'column'] + [f'eigenvector_{i}' for i in range(len(df.columns))]].groupby(['date', 'column']).mean().reset_index().sort_values('date').set_index('date')
    for i in range(len(df.columns)):
        print(f'eig_{i}', method)
        
        plt.figure(figsize=(10, 4))
        sns.lineplot(data=tmp, x='date', y='eigenvector_' + str(i), hue='column')
        plt.title(f'eigenvector_{i+1}')
        plt.show()

        plt.figure(figsize=(10, 4))
        sns.lineplot(data=tmp, x='date', y='eigenvector_' + str(i), hue='column')
        plt.title(f'eigenvector_{i+1}')
        plt.savefig(f'fig/eigenvector_{i+1}_{method}.png')
        plt.close()

In [ ]:
tmp = result.drop(columns=['column']).groupby(['date', 'method']).sum().reset_index().sort_values('date')
tmp['pl'] = 1 + tmp['pl'] / 100
tmp['pl'] = tmp.groupby('method')['pl'].cumprod()

plt.figure(figsize=(10, 4))
sns.lineplot(data=tmp, x='date', y='pl', hue='method')
plt.title('pl')
plt.show()

plt.figure(figsize=(10, 4))
sns.lineplot(data=tmp, x='date', y='pl', hue='method')
plt.title('pl')
plt.savefig(f'fig/pl_{method}.png')
plt.close()